In [ ]:
import yaml
import cv2
import numpy as np
import PIL.Image
import time
from pynq.lib.video import *
from pynq.overlays.base import BaseOverlay

In [ ]:
## Formate output frame for HDMI output ##

def format_frame(hdmi_out, frame):
    
    # Read input frame size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
    #print(frame.shape)
    
    # Creat new video formate frame
    outframe = hdmi_out.newframe()
    
    # Fill new frame with zero
    zero_img = np.zeros((480, 640), dtype=np.uint8)
    outframe[0:480,0:640] = zero_img[0:480,0:640]

    # Transfer array to the video frame
    # For Gray imge
    outframe[0:img_h,0:img_w] = frame[0:img_h,0:img_w]
    # For RBG image
    #outframe[0:480,0:640,:] = edges[0:480,0:640,:]
    
    return outframe

In [ ]:
## Line Detection ##

def line_detection(roi_rate, frame):
    
    # Read input img size
    img_h = frame.shape[0]
    img_w = frame.shape[1]
    
    # Take lower part of the frame
    roi = frame[int(img_h*roi_rate):, :]
    
    # Canny detect edges
    edges = cv2.Canny(roi, 39, 130)

    # Hough Line detect lines, output lines (x1, y1, x2, y2)
    lines = cv2.HoughLinesP(
        edges,
        rho=1.0, # The resolution of the parameter r in pixels. We use 1 pixel.
        theta=np.pi/180, # The resolution of the parameter θ in radians. We use 1 degree (CV_PI/180)
        threshold=8, # The minimum number of intersections to "*detect*" a line
        minLineLength=10, # The minimum number of points that can form a line. Lines with less than this number of points are disregarded.
        maxLineGap=9 # The maximum gap between two points to be considered in the same line.
    )
    
    # If no line detected
    if lines is None:
        # Return resized image
        overlay = frame
        
    # If line detected
    else:
        # draw lines
        line_img = np.zeros((img_h, img_w, 1), dtype=np.uint8)
        
        # center line
        cv2.line(line_img, (int(img_w/2), 0), (int(img_w/2), img_h), 100, 1)

        #for line in lines:
            #for x1, y1, x2, y2 in line:
                # Exclude lines that have a angle with the horizontal line less than 30 degree
                #if (abs(y2-y1) > 0.58 * abs(x2-x1)):
                    #cv2.line(line_img, (x1, y1+int(img_h*roi_rate)), (x2, y2+int(img_h*roi_rate)), 233, 1)
                    #cv2.circle(line_img, (x1, y1+int(img_h*roi_rate)), 3, 233, 0)
                    #cv2.circle(line_img, (x2, y2+int(img_h*roi_rate)), 3, 233, -1)
                    
        overlay = cv2.addWeighted(frame, 1, line_img, 1.0, 0.0)

    return lines, overlay

In [ ]:
## Find position offset from middle of two lines ##

def offset_from_middle(roi_rate, lines, overlay):

    if lines is None:
        return 144, overlay
    
    else:
        # Read input img size
        img_h = overlay.shape[0]
        img_w = overlay.shape[1]
        
        # Find the two lines that farthest to the middle 
        line_count = -1
        left_line_num = -1
        right_line_num = -1
        
        left_line_x1 = img_w/2
        right_line_x2 = img_w/2

        for line in lines:
            line_count = line_count + 1
            for x1, y1, x2, y2 in line:
                
                # Exclude lines that are less than 30 degree to the horizontal
                # tan(30) = 0.58
                if (abs(y2-y1) > 0.58 * abs(x2-x1)):
                    
                    # Find left line
                    if (x1 < img_w/2 and x1 < left_line_x1):
                        left_line_x1 = x1
                        left_line_num = line_count

                    # Find right line
                    if (x2 > img_w/2 and x2 > right_line_x2):
                        right_line_x2 = x2
                        right_line_num = line_count
                        
        # Output HDMI for Debug
        dot_img = np.zeros((img_h, img_w, 1), dtype=np.uint8)
        
        for x1, y1, x2, y2 in lines[left_line_num]:
            #y=kx+b
            left_b = x1

            #draw line
            cv2.line(dot_img, (x1, y1+int(img_h*roi_rate)), (x2, y2+int(img_h*roi_rate)), 233, 1)

            #draw dot
            cv2.circle(dot_img, (x1, y1+int(img_h*roi_rate)), 3, 233, 0)
            cv2.circle(dot_img, (x2, y2+int(img_h*roi_rate)), 3, 233, -1)


        for x1, y1, x2, y2 in lines[right_line_num]:
            #y=kx+b
            right_b = x2
            
            #draw line
            cv2.line(dot_img, (x1, y1+int(img_h*roi_rate)), (x2, y2+int(img_h*roi_rate)), 233, 1)
            
            #draw dot
            cv2.circle(dot_img, (x1, y1+int(img_h*roi_rate)), 3, 233, 0)
            cv2.circle(dot_img, (x2, y2+int(img_h*roi_rate)), 3, 233, -1)

        
        mid_dot = int((left_b + right_b)/2)
        cv2.circle(dot_img, (mid_dot, img_h), 3, 233, -1)
        
        overlay = cv2.addWeighted(overlay, 0.8, dot_img, 1.0, 0.0)
        
        return mid_dot, overlay

In [ ]:
## Initialize HDMI I/O ##

# Load the overlay
base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

# Configure HDMI input to gray scale ( (0.3 * R) + (0.59 * G) + (0.11 * B) )
hdmi_in.configure(PIXEL_GRAY)
hdmi_in.start()

# Configure Output resolution (w, h, bit per pixek)
hdmi_out_mode = VideoMode(640,480,8)
hdmi_out.configure(hdmi_out_mode, PIXEL_GRAY)
hdmi_out.start()

In [ ]:
## Read camera calibration parameter ##

fname = "calibration_parameter.yaml"
with open(fname) as file:
    data = yaml.load(file,Loader=yaml.Loader)
    
mtx = np.array([ [ data[0] , 0, data[1] ] , [ 0, data[2], data[3] ] , [0, 0, 1] ])
dist = np.array([ [ data[4], data[5], data[6], data[7], data[8] ] ])

In [ ]:
## Main ##

while(True):
    # Read frame from HDMI input
    frame = hdmi_in.readframe()
    
    # Resize frame (half of the 720p input)
    img = cv2.resize(frame, (640,360))
    
    # Undistort input frame
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    
    # Crop the frame baseo on undistortion
    x,y,w,h = roi
    #dst = dst[y:y+h, x:x+w]
    
    # Crop the frame to align center of the frame
    dst = dst[y:y+h, x+120:x+w-150]

    # ROI for line dection, the lower 1/4 of the frame
    roi_rate = 3/4

    # Dection line
    lines, lines_overlay = line_detection(roi_rate, dst)
    
    # Find lines offset to the middle of the frame
    mid_dot, offset_overlay = offset_from_middle(roi_rate, lines, lines_overlay)
    
    outframe = format_frame(hdmi_out, offset_overlay)
    
    hdmi_out.writeframe(outframe)   

In [ ]:
hdmi_out.stop()
hdmi_in.stop()
del hdmi_in, hdmi_out